### Imports

In [26]:
from tqdm import tqdm
from utils.utils import *

import json
import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.39 ms (started: 2023-07-12 13:28:47 -05:00)


### Parameters

In [27]:
dir_path = "output"

res = sorted(os.listdir(dir_path))
n = len(res) + 1
exp = "exp{}".format(n)
exp

'exp9'

time: 12.1 ms (started: 2023-07-12 13:28:47 -05:00)


In [28]:
method = "vgae" # vgae, dgi
dimension = 3
option = "" # dyn, str
raw_data_folder = "Edwin_proyecto3_v1" # change

time: 9.77 ms (started: 2023-07-12 13:28:47 -05:00)


### Load dataset

In [29]:
# load dataset groups
df1 = pd.read_csv("temp/{}/{}.csv".format(raw_data_folder, "Trial 1_Reinhard"), delimiter="|")


time: 29.1 ms (started: 2023-07-12 13:28:47 -05:00)


In [30]:
df1

,Alignment ID,Average Mz,Average Rt(min),Metabolite name,Starved1.1,Starved2.1,Starved3.1,Starved4.1,FCSglc1.1,FCSglc2.1,FCSglc3.1,FCSglc4.1,DMA1.1,DMA2.1,DMA3.1,DMA4.1
0,0,59.01425,3.909,Unknown,0.000310,0.000250,0.000540,0.000470,0.000300,0.000310,0.000240,0.000220,0.000250,0.000230,0.000240,0.000190
1,1,61.98612,1.811,Unknown,0.000220,0.000240,0.000230,0.000270,0.000150,0.000170,0.000170,0.000190,0.000160,0.000140,0.000180,0.000170
2,2,71.01184,3.639,Unknown,0.000031,0.000068,0.000054,0.000062,0.000180,0.000310,0.000230,0.000240,0.000210,0.000210,0.000220,0.000200
3,3,71.01184,3.511,Unknown,0.000078,0.000130,0.000068,0.000053,0.000250,0.000180,0.000190,0.000170,0.000260,0.000150,0.000230,0.000170
4,4,71.01184,3.765,Unknown,0.000041,0.000044,0.000034,0.000037,0.000170,0.000200,0.000180,0.000190,0.000170,0.000270,0.000190,0.000170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,3394,1178.40808,6.055,Unknown,0.000010,0.000005,0.000006,0.000005,0.000014,0.000028,0.000034,0.000027,0.000078,0.000017,0.000085,0.000005
3395,3395,1180.43262,4.818,Unknown,0.000002,0.000002,0.000002,0.000002,0.001720,0.001510,0.001630,0.001570,0.001710,0.000480,0.001670,0.001080
3396,3396,1185.20862,3.393,Unknown,0.000570,0.000005,0.000006,0.000005,0.000011,0.000015,0.000006,0.000004,0.000014,0.000006,0.000014,0.000009
3397,3397,1185.20862,3.592,Unknown,0.000007,0.000270,0.000300,0.000330,0.000007,0.000045,0.000030,0.000028,0.000005,0.000016,0.000035,0.000022


time: 20.5 ms (started: 2023-07-12 13:28:47 -05:00)


In [31]:
# drop duplicates
df1.drop_duplicates(subset=["Average Mz"], keep="last", inplace=True)
df1

,Alignment ID,Average Mz,Average Rt(min),Metabolite name,Starved1.1,Starved2.1,Starved3.1,Starved4.1,FCSglc1.1,FCSglc2.1,FCSglc3.1,FCSglc4.1,DMA1.1,DMA2.1,DMA3.1,DMA4.1
0,0,59.01425,3.909,Unknown,0.000310,0.000250,0.000540,0.000470,0.000300,0.000310,0.000240,0.000220,0.000250,0.000230,0.000240,0.000190
1,1,61.98612,1.811,Unknown,0.000220,0.000240,0.000230,0.000270,0.000150,0.000170,0.000170,0.000190,0.000160,0.000140,0.000180,0.000170
8,8,71.01184,4.864,Unknown,0.000042,0.000053,0.000042,0.000036,0.000130,0.000160,0.000110,0.000150,0.000047,0.000094,0.000130,0.000100
10,10,74.00508,1.492,Unknown,0.000015,0.000021,0.000008,0.000013,0.000800,0.000910,0.000590,0.000670,0.001010,0.000940,0.000750,0.001160
14,14,74.02387,5.895,Unknown,0.000034,0.000039,0.000043,0.000044,0.000130,0.000043,0.000110,0.000100,0.000039,0.000110,0.000059,0.000040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,3393,1177.92834,5.973,Unknown,0.000010,0.000010,0.000009,0.000009,0.000006,0.000011,0.000006,0.000004,0.000018,0.000005,0.000011,0.000092
3394,3394,1178.40808,6.055,Unknown,0.000010,0.000005,0.000006,0.000005,0.000014,0.000028,0.000034,0.000027,0.000078,0.000017,0.000085,0.000005
3395,3395,1180.43262,4.818,Unknown,0.000002,0.000002,0.000002,0.000002,0.001720,0.001510,0.001630,0.001570,0.001710,0.000480,0.001670,0.001080
3397,3397,1185.20862,3.592,Unknown,0.000007,0.000270,0.000300,0.000330,0.000007,0.000045,0.000030,0.000028,0.000005,0.000016,0.000035,0.000022


time: 30.7 ms (started: 2023-07-12 13:28:47 -05:00)


### Format dataset

In [32]:
# concat
df_join_raw = pd.concat([
    df1.iloc[:, 1:]], axis=1)
df_join_raw.set_index("Average Mz", inplace=True)

print(df_join_raw.shape)
df_join_raw = df_join_raw.rename_axis(None)
df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

(2494, 14)


,Starved1.1,Starved2.1,Starved3.1,Starved4.1,FCSglc1.1,FCSglc2.1,FCSglc3.1,FCSglc4.1,DMA1.1,DMA2.1,DMA3.1,DMA4.1
59.01425,0.000310,0.000250,0.000540,0.000470,0.000300,0.000310,0.000240,0.000220,0.000250,0.000230,0.000240,0.000190
61.98612,0.000220,0.000240,0.000230,0.000270,0.000150,0.000170,0.000170,0.000190,0.000160,0.000140,0.000180,0.000170
71.01184,0.000042,0.000053,0.000042,0.000036,0.000130,0.000160,0.000110,0.000150,0.000047,0.000094,0.000130,0.000100
74.00508,0.000015,0.000021,0.000008,0.000013,0.000800,0.000910,0.000590,0.000670,0.001010,0.000940,0.000750,0.001160
74.02387,0.000034,0.000039,0.000043,0.000044,0.000130,0.000043,0.000110,0.000100,0.000039,0.000110,0.000059,0.000040
...,...,...,...,...,...,...,...,...,...,...,...,...
1177.92834,0.000010,0.000010,0.000009,0.000009,0.000006,0.000011,0.000006,0.000004,0.000018,0.000005,0.000011,0.000092
1178.40808,0.000010,0.000005,0.000006,0.000005,0.000014,0.000028,0.000034,0.000027,0.000078,0.000017,0.000085,0.000005
1180.43262,0.000002,0.000002,0.000002,0.000002,0.001720,0.001510,0.001630,0.001570,0.001710,0.000480,0.001670,0.001080
1185.20862,0.000007,0.000270,0.000300,0.000330,0.000007,0.000045,0.000030,0.000028,0.000005,0.000016,0.000035,0.000022


time: 17.8 ms (started: 2023-07-12 13:28:47 -05:00)


In [33]:
# get groups name
print(df_join_raw.columns.values)

['Starved1.1' 'Starved2.1' 'Starved3.1' 'Starved4.1' 'FCSglc1.1'
 'FCSglc2.1' 'FCSglc3.1' 'FCSglc4.1' 'DMA1.1' 'DMA2.1' 'DMA3.1' 'DMA4.1']
time: 2.25 ms (started: 2023-07-12 13:28:47 -05:00)


In [34]:
# copy the group
groups_id = ["Starved", "FCSglc", "DMA"]

time: 7.47 ms (started: 2023-07-12 13:28:47 -05:00)


In [35]:
# change columns name (AB1.1)

""" columns = df_join_raw.columns.values

for i in range(len(columns)):
    # remove white space
    columns[i] = columns[i].replace(" ", "")
    columns[i] = columns[i].split("/")[1] 

df_join_raw.columns = columns
df_join_raw """

' columns = df_join_raw.columns.values\n\nfor i in range(len(columns)):\n    # remove white space\n    columns[i] = columns[i].replace(" ", "")\n    columns[i] = columns[i].split("/")[1] \n\ndf_join_raw.columns = columns\ndf_join_raw '

time: 5.51 ms (started: 2023-07-12 13:28:47 -05:00)


In [36]:
# get subgroups names
subgroups_id = get_subgroups_id(df_join_raw, groups_id)
subgroups_id

{'Starved': ['1', '2', '3', '4'],
 'FCSglc': ['1', '2', '3', '4'],
 'DMA': ['1', '2', '3', '4']}

time: 7.68 ms (started: 2023-07-12 13:28:47 -05:00)


In [37]:
# get options
options = {}

for group in groups_id:
    options[group] = [option]

time: 1.53 ms (started: 2023-07-12 13:28:47 -05:00)


### Save dataset and parameters

In [38]:
# save dataset
df_join_raw.to_csv("input/{}_raw.csv".format(exp), index=True)

# save parameters

# Data to be written
parameters = {
    "raw_folder": raw_data_folder,
    "exp": exp,
    "method": method,
    "dimension": dimension,
    "groups_id": groups_id,
    "subgroups_id": subgroups_id,
    "option": option
}
 
json_object = json.dumps(parameters)
with open("parameters.json", "w") as outfile:
    json.dump(parameters, outfile, indent=4)

time: 43.4 ms (started: 2023-07-12 13:28:47 -05:00)


### Create folders

In [39]:
# create experiments folder
try: 
    os.mkdir("output/{}".format(exp))
    os.mkdir("output/{}/preprocessing".format(exp))
    os.mkdir("output/{}/preprocessing/edges".format(exp))
    os.mkdir("output/{}/preprocessing/graphs".format(exp))
    os.mkdir("output/{}/preprocessing/graphs_data".format(exp))
    os.mkdir("output/{}/node_embeddings".format(exp))
    os.mkdir("output/{}/edge_embeddings".format(exp))
    os.mkdir("output/{}/common_edges".format(exp))
    os.mkdir("output/{}/plots".format(exp))
    os.mkdir("output/{}/biocyc".format(exp))
except OSError as error: 
    print(error)

time: 3.09 ms (started: 2023-07-12 13:28:47 -05:00)
